In [1]:
import cv2
import os
import pandas as pd

In [2]:
clicks = []
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Clicked at: ({x}, {y})")
        clicks.append((x, y))

def show_first_frame(name):
    '''
    Shows the first frame from a video
    Name is the video name
    '''

    window_name = 'find points'
    
    cap = cv2.VideoCapture(target + name + '.mp4')

    # resize window
    initial_width = 1920//2
    initial_height = 1080//2

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        
    # Calculate screen resolution
    screen_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    screen_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    
    # Calculate window position to center it on the screen
    window_x = int((screen_width - initial_width) / 2)
    window_y = int((screen_height - initial_height) / 2) - 200
    
    # Set window position
    cv2.moveWindow(window_name, window_x, window_y)
    
    cv2.resizeWindow(window_name, initial_width, initial_height)

    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, click_event)

    # set ot first frame
    cap.set(1,0)
    ret, img = cap.read()

    cv2.imshow(window_name, img)
    prev_clicks = len(clicks)
    print(prev_clicks)
    while cap.isOpened():
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        if len(clicks) == 6:
            break
        prev_clicks = len(clicks)

    cap.release()
    

In [3]:
target = 'target/lets/'
video = "Squash Refereeing： Ali Farag v Mohamed Abouelghar - Yes let [hMTkWkz8nRs]0"
show_first_frame(video)
cv2.destroyAllWindows()

0
Clicked at: (417, 332)
Clicked at: (865, 332)
Clicked at: (331, 479)
Clicked at: (961, 487)
Clicked at: (287, 552)
Clicked at: (995, 559)


In [4]:
parent_folder = 'target/'
subfolders = [folder for folder in os.listdir(parent_folder)]
columns = ['top left', 'top right', 'serv left', 'serv right', 'box left', 'box right', 'name']
df = pd.DataFrame(columns = columns)
clicks = []
for folder in subfolders:
    target = 'target/' + folder +'/'
    print(target)
    # List all files in the current subfolder
    files = os.listdir(target)
    for f in files:
        name = f.split('.')[0]
        show_first_frame(name)
        clicks.append(name)
        #new_df = pd.DataFrame([clicks], columns=columns)
        df = pd.concat([df, new_df], ignore_index=True)
        #print(df)
        print(clicks)
        clicks = []
        
cv2.destroyAllWindows()

target/lets/
0


NameError: name 'new_df' is not defined

In [81]:
df.to_csv('court_pos.csv', index=False)

In [5]:
cv2.destroyAllWindows()

In [82]:
df2 = pd.read_csv('court_pos.csv')

In [131]:
df2.head()

,top left,top right,serv left,serv right,box left,box right,name
0,"(415, 333)","(861, 336)","(329, 484)","(952, 487)","(288, 552)","(992, 560)",Squash Refereeing： Ali Farag v Mohamed Abouelg...
1,"(417, 331)","(861, 336)","(328, 484)","(953, 487)","(285, 552)","(996, 559)",Squash Refereeing： Asal v James - Yes Let [YEb...
2,"(344, 429)","(936, 431)","(257, 576)","(1015, 571)","(220, 635)","(1052, 632)",Squash Refereeing： Coll v Farag - Yes Let [YcA...
3,"(417, 331)","(861, 331)","(325, 480)","(951, 484)","(283, 552)","(991, 557)",Squash Refereeing： Coll v Mueller - Yes Let [A...
4,"(411, 347)","(871, 348)","(321, 501)","(960, 500)","(279, 573)","(1000, 575)",Squash Refereeing： Declan James v Mostafa Asal...


In [132]:
df2[df2['name'] == 'Squash Refereeing： Sobhy v King - Yes Let [Fhuz2FIDcz4]0']

,top left,top right,serv left,serv right,box left,box right,name
10,"(343, 432)","(935, 429)","(259, 577)","(1019, 572)","(223, 636)","(1052, 632)",Squash Refereeing： Sobhy v King - Yes Let [Fhu...


In [97]:
def get_x(x):
    x = x.split(',')
    return int(x[0][1:])

def get_y(x):
    x = x.split(',')
    return int(x[1][:-1])

In [121]:
def get_x_y(cols, name):
    x = []
    y = []
    row = df2.loc[df2['name'] == name].index[0]
    for col in cols:
        x.append(get_x(df2[col][row]))
        y.append(get_y(df2[col][row]))
    return x, y

In [110]:
# top left top right mid left mid right
corners = ['top left', 'top right', 'serv left', 'serv right']
cornersX = []
cornersY = []
for corner in corners:
    print(df2[corner][0])
    cornersX.append(get_x(df2[corner][0]))
    cornersY.append(get_y(df2[corner][0]))




(415, 333)
(861, 336)
(329, 484)
(952, 487)


In [128]:
cornersX, cornersY = get_x_y(corners, 'test0')
serv_boxX, serv_boxY = get_x_y(['serv left', 'serv right', 'box left', 'box right'], 'test0')
serv_frontX, serv_frontY = get_x_y(['top left', 'top right', 'box left', 'box right'], 'test0')

In [130]:
serv_boxX

[458, 1482, 366, 1568]

In [ ]:
front_serv = [df2[corner].apply(get_x) for corner in cornersX]
tly = df2['top left'].apply(get_y)